## Notebook Setup

In [ ]:
!pip install open3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

bunny.pcd    output			     TestingSuite.ipynb
bunny.pcd.1  python-pcl-0.2.0		     TOD.rar
data	     README.md			     tools
experiments  requirement.txt		     TrainOnNewImages.ipynb
lib	     result_2018-08-23-11-34-44.pcd  zero_waste.ipynb
LICENSE.md   ros			     ZeroWasteTestingSuite.ipynb


In [ ]:
import os
import sys

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ROB 498/UnseenObjectClustering'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))


sys.path.append("/content/" + GOOGLE_DRIVE_PATH)
sys.path.append("/content/" + GOOGLE_DRIVE_PATH + "/lib")
# sys.path.append(GOOGLE_DRIVE_PATH + "/lib/fcn")

['TestingSuite.ipynb', 'result_2018-08-23-11-34-44.pcd', 'README.md', 'LICENSE.md', '.gitignore', 'experiments', 'lib', '.git', 'tools', 'ros', 'data', 'requirement.txt', 'TOD.rar', 'output', 'python-pcl-0.2.0', 'bunny.pcd', 'bunny.pcd.1', 'TrainOnNewImages.ipynb', 'zero_waste.ipynb', 'ZeroWasteTestingSuite.ipynb']


In [ ]:
%cd "/content/drive/My Drive/ROB 498/UnseenObjectClustering"

/content/drive/My Drive/ROB 498/UnseenObjectClustering


In [ ]:
!pip uninstall -y pyyaml
!pip install transforms3d pillow IPython matplotlib easydict future scipy imageio
!pip uninstall -y pcl
!pip install pyyaml==5.4.1

Found existing installation: PyYAML 6.0
Uninstalling PyYAML-6.0:
  Successfully uninstalled PyYAML-6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for transforms3d: filename=transforms3d-0.4.1-py3-none-any.whl size=1376772 sha256=5030f4932e118bccc7c48a91ddc9121c8d0ae57823449f0632c39470bd6d9cba
  Stored in directory: /root/.cache/pip/wheels/a5/2d/e4/5d7de7a69ab4e6301b3125441725d47fe25b9e60e7d670ad21
Successfully built transforms3d
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 kB 23.8 MB/s eta 0:00:00


In [ ]:
## NOTE - we're not using PCL anymore since like every other file includes it we're including it here
!pip install pcl
import pcl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pcl: filename=pcl-0.0.0.post1-py3-none-any.whl size=1714 sha256=fb01ba5d004985fb39f026c42c67cd4642716f8873d80dc98df3bed58148548f
  Stored in directory: /root/.cache/pip/wheels/7d/2d/65/ff45617f0cbe370ffa3d78ea0a232bf4d29764af74616b54fc
Successfully built pcl


In [ ]:
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data

import argparse
import pprint
import time, os, sys
import os.path as osp
import numpy as np
import random
import scipy.io

#import _init_paths
from fcn.test_dataset import test_segnet
from fcn.config import cfg, cfg_from_file, get_output_dir
from datasets.factory import get_dataset
import networks

tabletop_object_train
tabletop_object_test
tabletop_object_all
osd_object_test
ocid_object_test


## Helper Functions

In [ ]:
def parse_args():
    """
    Parse input arguments
    """
    parser = argparse.ArgumentParser(description='Test a Unseen Clustering Network')
    parser.add_argument('--gpu', dest='gpu_id', help='GPU id to use',
                        default=0, type=int)
    parser.add_argument('--pretrained', dest='pretrained',
                        help='initialize with pretrained checkpoint',
                        default=None, type=str)
    parser.add_argument('--pretrained_crop', dest='pretrained_crop',
                        help='initialize with pretrained checkpoint for crops',
                        default=None, type=str)
    parser.add_argument('--cfg', dest='cfg_file',
                        help='optional config file', default=None, type=str)
    parser.add_argument('--dataset', dest='dataset_name',
                        help='dataset to train on',
                        default='shapenet_scene_train', type=str)
    parser.add_argument('--rand', dest='randomize',
                        help='randomize (do not use a fixed seed)',
                        action='store_true')
    parser.add_argument('--network', dest='network_name',
                        help='name of the network',
                        default=None, type=str)

    if len(sys.argv) == 1:
        parser.print_help()
        sys.exit(1)

    args = parser.parse_args()
    return args

## Configure Parameters for the model

In [ ]:
#TODO DREW --> Make a dictionary so we can easily test different models

#Artifically make the sys.argv good for parsing
sys.argv = ['./tools/test_net.py', '--network', 'seg_resnet34_8s_embedding', '--pretrained', 'data/checkpoints/seg_resnet34_8s_embedding_cosine_color_sampling_epoch_16.checkpoint.pth', '--dataset', 'osd_object_test', '--cfg', 'experiments/cfgs/seg_resnet34_8s_embedding_cosine_color_tabletop.yml', '--pretrained_crop', 'data/checkpoints/seg_resnet34_8s_embedding_cosine_color_crop_sampling_epoch_16.checkpoint.pth' ]

args = parse_args()

print('Called with args:')
print(args)

if args.cfg_file is not None:
    cfg_from_file(args.cfg_file)

if len(cfg.TEST.CLASSES) == 0:
    cfg.TEST.CLASSES = cfg.TRAIN.CLASSES
print('Using config:')
pprint.pprint(cfg)


if not args.randomize:
    # fix the random seeds (numpy and caffe) for reproducibility
    np.random.seed(cfg.RNG_SEED)


# device
cfg.gpu_id = 0
cfg.device = torch.device('cuda:{:d}'.format(cfg.gpu_id))
print('GPU device {:d}'.format(args.gpu_id))


Called with args:
Namespace(gpu_id=0, pretrained='data/checkpoints/seg_resnet34_8s_embedding_cosine_color_sampling_epoch_16.checkpoint.pth', pretrained_crop='data/checkpoints/seg_resnet34_8s_embedding_cosine_color_crop_sampling_epoch_16.checkpoint.pth', cfg_file='experiments/cfgs/seg_resnet34_8s_embedding_cosine_color_tabletop.yml', dataset_name='osd_object_test', randomize=False, network_name='seg_resnet34_8s_embedding')
Using config:
{'ANCHOR_RATIOS': [0.5, 1, 2],
 'ANCHOR_SCALES': [8, 16, 32],
 'BACKGROUND': '',
 'CAD': '',
 'DATA_PATH': '',
 'EPS': 1e-14,
 'EXP_DIR': 'tabletop_object',
 'FEATURE_STRIDE': 16,
 'FLIP_X': False,
 'FLOW_HEIGHT': 512,
 'FLOW_WIDTH': 640,
 'GPU_ID': 0,
 'INPUT': 'COLOR',
 'INTRINSICS': [],
 'MODE': 'TEST',
 'NETWORK': 'VGG16',
 'PIXEL_MEANS': array([[[102.9801, 115.9465, 122.7717]]]),
 'POSE': '',
 'RIG': '',
 'RNG_SEED': 3,
 'ROOT_DIR': '/content/drive/My Drive/ROB 498/UnseenObjectClustering',
 'TEST': {'ALIGN_Z_AXIS': False,
          'BBOX_REG': True,

## Prepare the Dataset

In [ ]:
#Parse through all the stuff on google drive
if cfg.TEST.VISUALIZE:
    shuffle = True
    np.random.seed()
else:
    shuffle = False
cfg.MODE = 'TEST'
dataset = get_dataset(args.dataset_name)
worker_init_fn = dataset.worker_init_fn if hasattr(dataset, 'worker_init_fn') else None
num_workers = 1
dataloader = torch.utils.data.DataLoader(dataset, batch_size=cfg.TEST.IMS_PER_BATCH, shuffle=shuffle,
    num_workers=num_workers, worker_init_fn=worker_init_fn)
print('Use dataset `{:s}` for training'.format(dataset.name))

111 images for dataset osd_object_test
Use dataset `osd_object_test` for training


## Prepare Network

In [ ]:
# overwrite camera intrinsics to get depth data
if len(cfg.INTRINSICS) > 0:
    K = np.array(cfg.INTRINSICS).reshape(3, 3)
    dataset._intrinsic_matrix = K
    print(dataset._intrinsic_matrix)

output_dir = get_output_dir(dataset, None)
print('Output will be saved to `{:s}`'.format(output_dir))
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Output will be saved to `/content/drive/My Drive/ROB 498/UnseenObjectClustering/output/tabletop_object/osd_object_test`


In [ ]:
# prepare network with pretrained model weights
if args.pretrained:
    network_data = torch.load(args.pretrained)
    if isinstance(network_data, dict) and 'model' in network_data:
        network_data = network_data['model']
    print("=> using pre-trained network '{}'".format(args.pretrained))
else:
    network_data = None
    print("no pretrained network specified")
    sys.exit()

network = networks.__dict__[args.network_name](dataset.num_classes, cfg.TRAIN.NUM_UNITS, network_data).cuda(device=cfg.device)
network = torch.nn.DataParallel(network, device_ids=[cfg.gpu_id]).cuda(device=cfg.device)
cudnn.benchmark = True

if args.pretrained_crop:
    network_data_crop = torch.load(args.pretrained_crop)
    network_crop = networks.__dict__[args.network_name](dataset.num_classes, cfg.TRAIN.NUM_UNITS, network_data_crop).cuda(device=cfg.device)
    network_crop = torch.nn.DataParallel(network_crop, device_ids=[cfg.gpu_id]).cuda(device=cfg.device)
else:
    network_crop = None

=> using pre-trained network 'data/checkpoints/seg_resnet34_8s_embedding_cosine_color_sampling_epoch_16.checkpoint.pth'
model keys
conv1.weight
bn1.weight
bn1.bias
bn1.running_mean
bn1.running_var
bn1.num_batches_tracked
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.bn1.num_batches_tracked
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.bn2.num_batches_tracked
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.bn1.running_mean
layer1.1.bn1.running_var
layer1.1.bn1.num_batches_tracked
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.bn2.running_mean
layer1.1.bn2.running_var
layer1.1.bn2.num_batches_tracked
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.bn1.running_mean
layer1.2.bn1.running_var
layer1.2.bn1.num_batches_tracked
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias

## Test the network

In [ ]:
# test network
test_segnet(dataloader, network, output_dir, network_crop)

Output hidden; open in https://colab.research.google.com to view.

## Visualize point clouds

In [ ]:
!pip install open3d

In [ ]:
!ls

data	     python-pcl-0.2.0		     TestingSuite.ipynb
experiments  README.md			     TOD.rar
lib	     requirement.txt		     tools
LICENSE.md   result_2018-08-23-11-34-44.pcd  TrainOnNewImages.ipynb
output	     ros			     zero_waste.ipynb


In [ ]:
import open3d as o3d
import numpy as np
#cloud = o3d.io.read_point_cloud("bunny.pcd")
cloud = o3d.io.read_point_cloud("result_2018-08-23-11-34-44.pcd", remove_nan_points=True)
cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

In [ ]:
points = np.asarray(cloud.points)

colors = None
if cloud.has_colors():
    colors = np.asarray(cloud.colors)
elif cloud.has_normals():
    colors = (0.5, 0.5, 0.5) + np.asarray(cloud.normals) * 0.5
else:
    geometry.paint_uniform_color((1.0, 0.0, 0.0))
    colors = np.asarray(geometry.colors)

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=1, color=colors)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
len(points)

269260